# Packages


In [9]:
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tqdm import tqdm
import json

# Get Lyrics


- API Schlüssel und Zugangsdaten einsetzen
- Playlist IDs von Spotify suchen 
- Jeder Track der Playlist wird auf LyricsGenius gesucht
- Sollte zu einem Song kein Lyrics gefunden werden, wird die Anzahl hochgezählt
- Es wird eine JSON Datei gebildet

In [6]:
SPOTIPY_CLIENT_ID = '46da6046625f409cb53ab06bf807861d'
SPOTIPY_CLIENT_SECRET = '00efb0ab68db4f1eaf27ec53a4c1e068'
GENIUS_ACCESS_TOKEN = 'OsBvEsJ_OLKYR42CJoaA_N97GNOS44XXFk4KPPcLtCYnWmDwnnzY2RpcG_SdG25a'

In [ ]:
playlist_id = '2siI2ILJEPr88yibWLgQQl'  #Playlist: "Deutsche Songs die jeder kennt ;)"

Ausführen von getlyrics.py

## Analysis

In [3]:
with open("./all_songs.json") as f:
    songs_dict = json.load(f)

1. Spacy

Ausführen von cyspa.py

In [21]:
#NEU #Spacy muss gedownloaded werden mit : pip install -U pip setuptools wheel   und    pip install spacy 
#de_core_news_sm-3.5.0-py3-none-any.whl (muss gedownloaded werden -> https://spacy.io/models/de/)
#de_core_news_lg-3.5.0-py3-none-any.whl
#de_dep_news_trf-3.5.0-py3-none-any.whl

nlp = spacy.load("de_core_news_sm")
#nlp1 = spacy.load ("de_dep_news_trf")
nlp2 = spacy.load ("de_core_news_lg")

list_of_nouns, list_of_pronouns = [], []

def spacify_with_coref(lyrics_from_song):
    doc3 = nlp2(lyrics_from_song)
    doc = nlp(lyrics_from_song)

    doc._.coref_chains = doc3._.coref_chains

    return doc

for song_id in tqdm(range(len(songs_dict["Prototyp"]))):

    lyrics_from_song = songs_dict["Prototyp"][str(song_id)]["lyrics"]
    doc = nlp(lyrics_from_song)
    #doc2 = nlp1 (lyrics_from_song)
    doc3 = nlp2 (lyrics_from_song)

    # Ausgabe lediglich von Wörtern, die Nomen und Pronomen sind:
    for token in doc3: 
        if token.pos_ == "NOUN":
            list_of_nouns.append(token.text)
        elif token.pos_ == "PRON":
            list_of_pronouns.append(token.text)
        else:
            pass

100%|██████████| 88/88 [00:23<00:00,  3.77it/s]


In [17]:
print(f"{len(list_of_nouns)} nouns were found.")
print(f"{len(list_of_pronouns)} pronouns were found.")

6516 nouns were found.
5504 pronouns were found.


In [19]:
distinct_nouns = sorted(list(set(list_of_nouns)))
distinct_pronouns = sorted(list(set(list_of_pronouns)))
print(f"{len(distinct_nouns)} distinct nouns were found.")
print(f"{len(distinct_pronouns)} distinct pronouns were found.")

2169 distinct nouns were found.
110 distinct pronouns were found.


In [20]:
distinct_nouns

['.',
 '12Embed',
 '1687.',
 '1Embed',
 '257ers',
 '36grad',
 '4Embed',
 '70s',
 'A',
 'A-N-N-A',
 'A.C.A.',
 'AC—DC',
 'AFFE',
 'AFTER',
 'AGUILERA',
 "AIN'T",
 'ALEJANDRO',
 'ALICIA',
 'ALIVE',
 'ALLES',
 'ALOE',
 'ALTES',
 'AM',
 'AMSTERDAM',
 'ANIMALS',
 'ANTOINE',
 'APPLAUSE',
 'ARMY',
 'AROUND',
 'ARZT',
 'ASCHENFLUG',
 'AVICII',
 'AWAY',
 'Aas',
 'Abend',
 'Abendbrot',
 'Abrissbirne',
 'Abstand',
 'Achter',
 'Achterbahn',
 'Adapted',
 'Adel',
 'Adern',
 'Affe',
 'Affekt',
 'Affen',
 'Affenpower',
 'Affer',
 'Aggro',
 'Ah',
 'Akten',
 'Albumversion',
 'Alkohol',
 'All',
 'Alligatoah',
 'Alltag',
 'Alter',
 'Altlast',
 'Amoklauf',
 'Anblick',
 'Andern',
 'Anfang',
 'Anglo-Chinese',
 'Angst',
 'Ansage',
 'Anschlag',
 'Antrieb',
 'Anwalt',
 'Applaus',
 'Apuleius',
 'Arie',
 'Arm',
 'Armee',
 'Armen',
 'Arsch',
 'Arschgeweih',
 'Art',
 'Arzt',
 'Asche',
 'Ascheregen',
 'Asi',
 'Asphalt',
 'Ast',
 'Astronaut',
 'Astronaut8Embed',
 'Atem',
 'Atemlos',
 'Atemzug',
 'Atme',
 'Atmen',
 'A